In [33]:
'''
Steps
1. installed spacy and download the language model
2. install nltk - this can be used to tokenize
'''
%pip install -U spacy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Implement NER with spacy sample test
import spacy

# Load the spacy english model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "Apple Inc. is a technology company based in Cupertino, California. It was founded by Steve Jobs."

# Process the text with spaCy, this does tokenization for each 
# word(tells the text, part-of-speech, and dependency relation)
# Ex: Apple PROPN compound
doc = nlp(text)

# testing what doc gives me
# for token in doc:
#     print(token.text, token.pos_, token.dep_, token.ent_iob_)

# Print the named entities in the text
for ent in doc.ents:
    print(f"{ent.text} - {ent.label_}")

    



Apple Inc. - ORG
Cupertino - GPE
California - GPE
Steve Jobs - PERSON


In [34]:
# We want the named entities to be: PERSON, LOC, ORG, OTHER so using spacy we can train a model to work with
# those entities, lets first give some training_data

import spacy
from spacy.training.example import Example

training_data = [
    Example.from_dict(nlp.make_doc("Apple is a software engineering company."), {"entities" : [(0, 5, "ORG")]}),
    Example.from_dict(nlp.make_doc("Shubham works at Apple in Maryland."), {"entities": [(0, 7, "PERSON"), (26, 34, "LOC")]}),
]

# load the spacy model and NER component that we will train this data with
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# add the label_tags to the ner we are creating
label_tags = ["PERSON", "ORG", "LOC", "OTHER"]
for x in label_tags:
    ner.add_label(x)

# Disable other pipeline components during training for efficiency
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    # Training loop
    for epoch in range(10):  # Adjust the number of epochs as needed
        for example in training_data:
            nlp.update([example], drop=0.5)  # Adjust the dropout parameter

# Save the trained model
nlp.to_disk("custom_ner_model")



# text = "Apple Inc. is a technology company based in Cupertino, California. It was founded by Steve Jobs."
# doc = nlp(text)



KeyError: "[E022] Could not find a transition with the name 'O' in the NER model."

In [35]:
import spacy
from spacy.training.example import Example

# Create a blank English model
nlp = spacy.blank("en")

# Add NER component to the pipeline
ner = nlp.add_pipe("ner")

# Define your custom labels
custom_labels = ["PERSON", "LOC", "ORG", "OTHER"]

# Add custom labels to NER
for label in custom_labels:
    ner.add_label(label)

# Example training data
TRAIN_DATA = [
    ("John works at XYZ Corp.", {"entities": [(0, 4, "PERSON"), (15, 22, "ORG")]}),
    ("Paris is a beautiful city.", {"entities": [(0, 5, "LOC")]}),
    # Add more examples as needed with PERSON, LOC, ORG, and OTHER entities
]

# Prepare training data as Example objects
train_examples = []
for text, annotations in TRAIN_DATA:
    train_examples.append(Example.from_dict(nlp.make_doc(text), annotations))

# Disable other pipeline components during training for efficiency
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    # Training loop
    for epoch in range(10):  # Adjust the number of epochs as needed
        for example in train_examples:
            nlp.update([example], drop=0.5)  # Adjust the dropout parameter

# Save the trained model
nlp.to_disk("custom_ner_model")


/Users/shubham/anaconda3/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "John works at XYZ Corp." with entities "[(0, 4, 'PERSON'), (15, 22, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


KeyError: "[E022] Could not find a transition with the name 'O' in the NER model."